# Analyse point source density 

In [ ]:
%matplotlib inline

import re
import os
import sys
import glob
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from rtree import index

sys.path.append('/Users/mpagani/Repos/original/oq-engine/')
sys.path.append('/Users/mpagani/Repos/oq-man/')

import oqman.man.checking_utils.source_model_utils as model
from oqman.man.single_source_utils.areas_utils import get_rates_density
from oqman.man.single_source_utils.points_utils import get_cell_areas, get_rates_density

from openquake.hazardlib.const import TRT
from openquake.hazardlib.source import PointSource

## List of models to be analysed

In [ ]:
path = '/Users/mpagani/NC/Hazard_Charles/Documents/Presentations/2017/0417_SSA/notebooks/pkl/*.pkl'
modell = set(['als07', 'aus12', 'ca10h', 'ca10r', 'cub03', 'ear16', 'em16a',
              'em16f', 'emc15', 'lea02', 'nzl10', 'res12', 'sar16', 'sea07',
              'soa10', 'twn15', 'usa08'])
#modell = set(['cub03'])
#modell = set(['nzl10'])
#modell = set(['usa08'])
#modell = set(['als07'])
modell = set(['nzl10', 'soa10'])
modell = set(['soa10'])
modell = set(['sar16'])
modell = set(['em16f'])
modell = set(['usa08'])
#modell = set(['nzl10'])

### Creating the pickle file with the size of each cell

In [ ]:
modd = {}
for fname in glob.glob(path):
    slist = re.split('\.',  os.path.basename(fname))
    if re.search('[a-z]*[0-9]*\.pkl', fname) and slist[0] in modell:
        print ('>>', fname, end=' ')
        # Creating output filename 
        dname = os.path.dirname(fname)
        slist = re.split('_', re.split('\.',  os.path.basename(fname))[0])
        outf = os.path.join(dname, '{0:s}_cellsize_sources.pkl'.format(slist[0]))
        if os.path.exists(outf):
            print ('loading ', outf)
            fou = open(outf, 'rb')
            csize = pickle.load(fou)
            print (len(csize))
            fou.close()
            modd[slist[0]] = {'csize': csize}
        else:
            print ('skipping')

## Read data for the different models

In [ ]:
%%time
for fname in glob.glob(path):
    slist = re.split('\.',  os.path.basename(fname))
    if re.search('[a-z]*[0-9]*\.pkl', fname) and slist[0] in modell:
        print ('>>', fname, end=' ')
        mod, point, info, sidx = model.load(fname)      
        print (len(point), end=' ')
        print ('done')
        if slist[0] not in modd:
            modd[slist[0]] = {}
        modd[slist[0]]['model'] = mod
        modd[slist[0]]['info'] = info
        modd[slist[0]]['point'] = point
        modd[slist[0]]['sidx'] = sidx

## Computing the area of cells 

In [ ]:
cnt = 1
for key in sorted(modd.keys()):
    #
    # Check if the model contains point sources
    if 'PointSource' in modd[key]['info']['srcs_mmax']:
        print ('>>>>>>>>', key)
        #
        # set parameters
        fltr = set(['Active Shallow Crust']); 
        sxx = 0.35
        #
        # create points list
        cnt = 0
        pnts = []
        lons = []
        lats = []
        for idx, pnt in enumerate(modd[list(modell)[0]]['point']):
            if isinstance(pnt, PointSource):
                lon = pnt.location.longitude
                lat = pnt.location.latitude
                dep = pnt.location.depth
                lons.append(lon)
                lats.append(lat)
                pnts.append(pnt)
                cnt += 1

        #
        # compute area of grid cells 
        aaa, coloc, coo, sidx = get_cell_areas(pnts)
        print ('Area calculation completed')
        jjj = np.isnan(aaa)
        print (np.any(jjj))
        #
        # plot 
        fig = plt.figure(figsize=(8,10))
        plt.plot(coo[:,0], coo[:,1], 's') 
        plt.plot(coo[:,0][jjj], coo[:,1][jjj], 'or', zorder=100)
        # updating counter
        cnt += 1
        #plt.xlim([170, 180])
        """
        """

In [ ]:
# computing density
dend, _, cidx, scoo = get_rates_density(pnts, mmint=5.5)
print (len(cidx), len(dend))

In [ ]:
scoo = np.array(scoo)
print (sorted(set(scoo[:,2])))

In [ ]:
# plot 
import matplotlib
fig = plt.figure(figsize=(8,10))
iii = np.nonzero(scoo[:,2] < 15.0)
print (iii[0].dtype)
dend = np.array(dend)
plt.scatter(scoo[iii[0],0], coo[iii[0],1], c=dend[iii[0]], norm=matplotlib.colors.LogNorm()) 

In [ ]:
print (np.amin(dend))
print (np.amax(dend))

In [ ]:
#
# settings
import matplotlib as mpl
mpl.rcParams['xtick.labelsize'] = 14
mpl.rcParams['ytick.labelsize'] = 14
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['axes.titlesize'] = 16
#
# figure 
fig = plt.figure(figsize=(10,8))
ax = plt.subplot(111)
bins = np.logspace(-10, -4, 25)
his, _ = np.histogram(dend, bins=bins)
his = his / sum(his)
plt.bar(bins[:-1], his, edgecolor='black', width=np.diff(bins)*0.92, align='edge')
plt.xscale('log')
plt.xlabel('Eqk rate [1/(km^2*yr)]')
ax.set_title(list(modell)[0])

##### 